## First look at the dataset in the last project
### Passive acoustic emission during the feedback-controlled rock core deformation experiment
### Nov, 22, 2023, Edinburgh
CL04 is constant strain rate loading

------------------------------------------
Acoustic emission recordings:

/CL04_AE/
bottom_sensor_AE_CL04.ascii % recordings from bottom receiver / 16384 rows (individual waveform time-series with dt=2e-8s) vs. 2235 columns (individual AE)
top_sensor_AE_CL04.ascii    % recordings from top receiver / 16384 rows (individual waveform time-series with dt=2e-8s) vs. 2235 columns (individual AE)
times_sec_from_loading_AE_CL04.ascii % timing of individual AE as seconds from loading time (2235 entries)
Note: This is raw data, recorded at 60db pre-amp gain. To convert to 'equivalent' recording at 70db (as per CL10), multiply by sqrt(10)
(because going from 60 db to 70 db involves multiplication by 10 ^ (10/20)Â = sqrt(10) )

------------------------------------------
Estimated system delay through sample:
This estimate was based on an experiment whereby the pistons touched together (sample-less experiment)
2.6493e-05s to correct for pistons and for any external factor delay (recording system etc)





In [ ]:
#import some basic package

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
np.bool = np.bool_ #Such tricky is used to avoid over-updated numpy (https://stackoverflow.com/questions/74893742/how-to-solve-attributeerror-module-numpy-has-no-attribute-bool)

## let's use plotly, very useful, powerful new platform
import plotly.offline as pyo
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#express environment
import plotly.express as px

In [ ]:

path='CL04_seismic/CL04_AE/'

#first, define the shot time
load_times=pd.read_csv(path+'times_sec_from_loading_AE_CL04.ascii', header=None, delimiter=r"\s+")


#must set up time intervals
dt=2E-8

#sensor here!
b_s_a=pd.read_csv(path+'bottom_sensor_AE_CL04.ascii', header=None, delimiter=r"\s+")
b_s_a=b_s_a.set_index(dt*b_s_a.index)
b_s_a.columns = load_times.values[0]


t_s_a=pd.read_csv(path+'top_sensor_AE_CL04.ascii', header=None, delimiter=r"\s+")
t_s_a=t_s_a.set_index(dt*t_s_a.index)
t_s_a.columns = load_times.values[0]



In [ ]:
# t_s_a.iloc[:,2].plot()
# b_s_a.iloc[:,2].plot()
fig = go.Figure()
fig.add_trace(go.Scatter(x= t_s_a.iloc[:,1221].index, y= t_s_a.iloc[:,1221],
                    mode='lines',
                    name='top'))

fig.add_trace(go.Scatter(x= b_s_a.iloc[:,1221].index, y= b_s_a.iloc[:,1221],
                    mode='lines',
                    name='bottom'))

layout = go.Layout(
    title='Top/Bottom Source:'+str(t_s_a.iloc[:,1221].name)+'s', title_x=0.5,
    xaxis=dict(
        title='Time (s)',
         ),
    yaxis=dict(title="Amplitude"))

fig.update_layout(layout, width=600, height=400)


In [ ]:
##let's do two-D color plot
fig = go.Figure()

dtsa=t_s_a.iloc[:,::30]

fig = px.imshow(dtsa, text_auto=True, aspect="auto",title = 'top source showing coda importance',labels={'x':'shot time (s)','y':'seismic time (s)'}, range_color=[-0.3, 0.3])

fig.update_layout(coloraxis_colorbar=dict(
    title="top source shift",
    thicknessmode="pixels", thickness=10,
    lenmode="pixels", len=200,
    yanchor="top", y=1,
    ticks="outside", 
    dtick=0.1
), width=800, height=600, )
#be careful, it is already reversed!
fig.update_yaxes(autorange=True, row=1, col=1)

fig.show()

##do not use the below, bad axis control
# fig.add_trace(go.Heatmap(
#     z=t_s_a,
#     colorbar=dict(
#         title="legend",
#         titleside="top",
#         tickmode="array",
#         #tickvals=[2, 25, 50, 75, 100],
#         #labelalias={100: "Hot", 50: "Mild", 2: "Cold"},
#         ticks="outside"
#     )
# ))

In [ ]:
#OK! to interpret this, we need to read the operation data in the lab
mechanical=pd.read_csv('CL04_mechanical.csv', index_col=0)
mechanical

In [ ]:
## let 's try to plot all the data here

fig = go.Figure()

#in this environment, we should do subtitles by ourself

fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                    vertical_spacing=0.1)

fig.add_trace(go.Heatmap(
    z=dtsa, x=dtsa.columns,zmin=-0.3,zmax=0.3,
    colorbar=dict(
        title="legend",
        titleside="top",
        tickmode="array",
        #tickvals=[2, 25, 50, 75, 100],
        #labelalias={100: "Hot", 50: "Mild", 2: "Cold"},
        ticks="outside"
    )
))


fig.add_trace(go.Scatter(x= mechanical.index, y= mechanical['differential stress (MPa)'],
                    mode='lines',
                    name=str(mechanical.iloc[:,0].name),fillcolor='red'),row=2, col=1)

fig.add_trace(go.Scatter(x= mechanical.index, y= mechanical['axial strain'],
                    mode='lines',
                    name=str(mechanical.iloc[:,1].name),fillcolor='blue'),row=3, col=1)


layout = go.Layout(
    title="mechanical operation", title_x=0.45,
     showlegend=False,
     xaxis=dict(
        title='Loading Time(s)'),
     yaxis=dict(title=""),
    legend_title_text='Snapshots'
         )

fig.update_layout(layout, autosize = False, width=800, height=1200)    
fig.show()

In [ ]:
# ##let's do two-D color plot
# fig = go.Figure()


# fig = px.imshow(b_s_a, text_auto=True, aspect="auto",title = 'bottom source showing coda importance',labels={'x':'shot time (s)','y':'seismic time (s)'}, range_color=[-0.3, 0.3])



# fig.update_layout(coloraxis_colorbar=dict(
#     title="top source shift",
#     thicknessmode="pixels", thickness=10,
#     lenmode="pixels", len=300,
#     yanchor="top", y=1,
#     ticks="outside", 
#     dtick=0.1
# ), width=800, height=600, )
# #be careful, it is already reversed!
# fig.update_yaxes(autorange=True, row=1, col=1)

# fig.show()

In [ ]:
# ## let 's try to plot all the data here

# fig = go.Figure()

# #in this environment, we should do subtitles by ourself

# subplot_titles=[]
# for i in range (0,18):
#     subplot_titles+=[str(t_s_a.iloc[:,i].name)+'s']



# fig = make_subplots(rows=18, cols=1, shared_xaxes=False,
#                     vertical_spacing=0.01,  subplot_titles=subplot_titles)

# for i in range (0,18):
#     fig.add_trace(go.Scatter(x= t_s_a.iloc[:,i].index, y= t_s_a.iloc[:,i],
#                     mode='lines',
#                     name='top'+str(t_s_a.iloc[:,i].name)+'s',fillcolor='red'),row=i+1, col=1)

#     fig.add_trace(go.Scatter(x= b_s_a.iloc[:,i].index, y= b_s_a.iloc[:,i],
#                     mode='lines',
#                     name='bottom'+str(b_s_a.iloc[:,i].name)+'s',fillcolor='blue'),row=i+1, col=1)


# layout = go.Layout(
#     title="Top/Bottom Source 18 snapshots", title_x=0.45,
#      showlegend=False,
#      xaxis=dict(
#         title='Time(s)'),
#      yaxis=dict(title="Aplitude"),
#     legend_title_text='Snapshots'
#          )

# fig.update_layout(layout, autosize = False, width=800, height=8000)    
# fig.show()